# Tests Blocked or Timed Out

This notebook is an extension to the [number_of_flakes](number_of_flakes.ipynb) notebook. In this notebook, the key perfomance indicators that we would like to create greater visbility into and track over time is the percent of tests that got blocked or were timed out. By observing the above metrics and tracking them wrt time, we will be able to quantify the efficiency of our testing platforms.

* number and percent of tests blocked
* number and percent of tests timed out

In [3]:
import gzip
import json
import os
import sys
import pandas as pd

sys.path.append("../../..")

module_path_1 = os.path.abspath(os.path.join("../../../data-sources/TestGrid"))
if module_path_1 not in sys.path:
    sys.path.append(module_path_1)

from ipynb.fs.defs.number_of_flakes import (  # noqa: E402
    testgrid_labelwise_encoding,
)  # noqa: E402

In [4]:
# Load test file
with gzip.open("../../../../data/raw/testgrid_183.json.gz", "rb") as read_file:
    testgrid_data = json.load(read_file)

In [5]:
blocked_tests_list = testgrid_labelwise_encoding(testgrid_data, 8)[0:1000000]

In [6]:
len(blocked_tests_list)

1000000

In [7]:
blocked_tests_list[0]

(datetime.datetime(2021, 3, 15, 23, 40, 20),
 '"redhat-assisted-installer"',
 'periodic-ci-openshift-release-master-nightly-4.6-e2e-metal-assisted',
 'Overall',
 [{'metric': ['test-duration-minutes'],
   'values': [[80.28333333333333,
     92.05,
     80.98333333333333,
     117.71666666666667,
     108.63333333333334,
     87.06666666666666,
     111.03333333333333,
     100.28333333333333,
     143.2,
     577.0333333333333,
     18.833333333333332,
     92.81666666666666,
     81.36666666666666,
     89.05,
     84.08333333333333,
     108.68333333333334,
     82.61666666666666,
     99.98333333333333,
     99.63333333333334,
     84.35,
     80.15]]}],
 False)

In [8]:
# Convert to dataframe
blocked_tests_df = pd.DataFrame(
    blocked_tests_list,
    columns=["timestamp", "tab", "grid", "test", "graph", "test_blocked"],
)
blocked_tests_df.head()

,timestamp,tab,grid,test,graph,test_blocked
0,2021-03-15 23:40:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
1,2021-03-15 00:01:06,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
2,2021-03-13 20:51:32,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
3,2021-03-13 07:51:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
4,2021-03-13 06:43:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False


In [9]:
# saving only the first 1000000 out of 19 million rows due to pvc limits.
# 250mb = 1 million --> 4750 mb = 19 million
blocked_tests_df.head(1000).to_csv(
    "../../../../data/processed/blocked_tests.csv",
    header=False,
)

In [10]:
timed_out_tests_list = testgrid_labelwise_encoding(testgrid_data, 9)[0:1000000]

In [11]:
# Convert to dataframe
timed_out_tests_df = pd.DataFrame(
    timed_out_tests_list,
    columns=["timestamp", "tab", "grid", "test", "graph", "test_timed_out"],
)
timed_out_tests_df.head()

,timestamp,tab,grid,test,graph,test_timed_out
0,2021-03-15 23:40:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
1,2021-03-15 00:01:06,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
2,2021-03-13 20:51:32,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
3,2021-03-13 07:51:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False
4,2021-03-13 06:43:20,"""redhat-assisted-installer""",periodic-ci-openshift-release-master-nightly-4...,Overall,"[{'metric': ['test-duration-minutes'], 'values...",False


In [12]:
# saving only the first 1000000 out of 19 million rows due to pvc limits.
# 250mb = 1 million --> 4750 mb = 19 million
timed_out_tests_df.head(1000).to_csv(
    "../../../../data/processed/timed_out_tests.csv",
    header=False,
)

In [13]:
# Metrics
no_tests = blocked_tests_df.head(1000000).test.count()
print("Total number of tests: %i" % (no_tests))
no_blocked = blocked_tests_df.head(1000000).test_blocked.sum()
print("Total number of tests blocked: %i" % (no_blocked))
test_blocked_percentage = (
    (
        blocked_tests_df.head(1000000).test_blocked.sum()
        / blocked_tests_df.head(1000000).test.count()
    )
) * 100
print("Tests blocked percentage: %f" % (test_blocked_percentage))
no_timed_out = timed_out_tests_df.head(1000000).test_timed_out.sum()
print("Total number of timed out tests: %i" % (no_timed_out))
test_timed_out_percentage = (
    (
        timed_out_tests_df.head(1000000).test_timed_out.sum()
        / timed_out_tests_df.head(1000000).test.count()
    )
) * 100
print("Test timed out percentage: %f" % (test_timed_out_percentage))

Total number of tests: 1000000
Total number of tests blocked: 0
Tests blocked percentage: 0.000000
Total number of timed out tests: 0
Test timed out percentage: 0.000000
